<center><h1>Module 8 (part B) - Word vectors and word embeddings</h1></center>

We are now ready to try out a KNN style of prediction using word vectors.

##Bring in your library now

You need the functions you defined in part A.

In [1]:
#your code
!rm -r  'w20_ds_library'
clone_url = f'https://github.com/FutureDeus/w20_ds_library.git'
!git clone $clone_url
from w20_ds_library import *

Cloning into 'w20_ds_library'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 120 (delta 76), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (120/120), 27.47 KiB | 1.53 MiB/s, done.
Resolving deltas: 100% (76/76), done.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Make sure you see your part A functions from below listing.

In [2]:
%whos

Variable                    Type                    Data/Info
-------------------------------------------------------------
Callable                    CallableMeta            typing.Callable
TypeVar                     TypingMeta              typing.TypeVar
addv                        function                <function addv at 0x7f6d04dd5c80>
bayes                       function                <function bayes at 0x7f6d04dd5840>
bayes_tester                function                <function bayes_tester at 0x7f6d04dd58c8>
char_set                    str                     #!abcdefghijklmnopqrstuvwxyz
clone_url                   str                     https://github.com/FutureDeus/w20_ds_library.git
cm_accuracy                 function                <function cm_accuracy at 0x7f6d04dd56a8>
cosine_similarity           function                <function cosine_similarity at 0x7f6d04dd5730>
dframe                      TypeVar                 ~pd.core.frame.DataFrame
dict_ordered_distances 

##Bring in spacy pieces

In [0]:
import spacy

In [4]:
!python -m spacy download en_core_web_md


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md
nlp = en_core_web_md.load()  

##Bring in the gothic_authors

In [0]:
import pandas as pd
gothic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQqRwyE0ceZREKqhuaOw8uQguTG6Alr5kocggvAnczrWaimXE8ncR--GC0o_PyVDlb-R6Z60v-XaWm9/pub?output=csv',
                          encoding='utf-8')

In [7]:
gothic_table.head()


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [8]:
len(gothic_table)

19579

Let's get shuffled version so we can do testing later.

In [0]:
import numpy as np
rsgen = np.random.RandomState(1234)  #use this so we get same values

In [0]:
shuffled_gothic = gothic_table.sample(frac=1, random_state=rsgen).reset_index(drop=True)

In [11]:
shuffled_gothic.head()

,id,text,author
0,id18824,"An indefinite sense of awe, which at first sig...",EAP
1,id27368,"Surely, man had never before so terribly alter...",EAP
2,id06142,"Why don't you laugh at Oliver's grandfather, w...",HPL
3,id25016,I lay upon the grass surrounded by a darkness ...,MWS
4,id09465,"She stood erect in a singularly fragile canoe,...",EAP


Pull out the sentences in raw form.

In [0]:
gothic_sentences = shuffled_gothic['text'].tolist()

In [13]:
gothic_sentences[1]  #'Surely, man had never before so terribly altered, in so brief a period, as had Roderick Usher It was with difficulty that I could bring myself to admit the identity of the wan being before me with the companion of my early boyhood.'

'Surely, man had never before so terribly altered, in so brief a period, as had Roderick Usher It was with difficulty that I could bring myself to admit the identity of the wan being before me with the companion of my early boyhood.'

#Challenge 5

##Ok, this is cool

We can revisit KNN but now using our sentence vectors as the crowd.

First thing: generate the list of sentence vectors, which will end up being the basis for our crowd. Took me 4 minutes.

In [14]:
%%time

#your code

sentence_vectors = [sent2vec(nlp, i) for i in gothic_sentences]


CPU times: user 3min 21s, sys: 331 ms, total: 3min 21s
Wall time: 3min 21s


In [15]:
len(sentence_vectors)  #19579

19579

In [16]:
type(sentence_vectors)

list

In [17]:
type(sentence_vectors[0])  #so a list of numpy vectors

numpy.ndarray

In [18]:
len(sentence_vectors[0])  #300 like we expect

300

In [19]:
sentence_vectors[0][:10] #array([ 0.06375549,  0.12829445, -0.10589062, -0.05812356, -0.00601913, 0.02335889, -0.08290436,  0.04431987, -0.08755204,  2.3948934 ], dtype=float32)

array([ 0.06375549,  0.12829445, -0.10589062, -0.05812356, -0.00601913,
        0.02335889, -0.08290436,  0.04431987, -0.08755204,  2.3948934 ],
      dtype=float32)

#Challenge 6

##Now need a new ordered distance function

It takes in a crowd, e.g., something like sentence_vectors, and a sentence as a raw string. It converts the string to a vector then uses fast_cosine to get a list of values. As you know, with cosine the bigger the better (opposite of distance with euclidean distance).

In [0]:
def vector_ordered_distances(nlp:spnlp, crowd:list, input_str:str) -> list:
  assert isinstance(crowd, list)
  assert all([isinstance(v, numpy.ndarray) for v in crowd])

  #your code here
  s2v = sent2vec(nlp, input_str)
  return sorted([(keys, fast_cosine(values, s2v)) for keys, values in enumerate(crowd)], key = lambda x: x[1], reverse = True)


##Test on row 0

In [39]:
ordered = vector_ordered_distances(nlp, sentence_vectors, gothic_sentences[0])  #note will compare against self so should get a 1 first
ordered[:10]

[(0, 0.99999994),
 (12799, 0.9675024),
 (18159, 0.96728075),
 (1297, 0.96718),
 (3206, 0.9665487),
 (2889, 0.9661672),
 (1133, 0.9657392),
 (8376, 0.96559393),
 (19369, 0.9654488),
 (17071, 0.9654202)]

<pre>
[(0, 0.99999994),
 (12799, 0.9675024),
 (18159, 0.96728075),
 (1297, 0.96718),
 (3206, 0.9665487),
 (2889, 0.9661672),
 (1133, 0.9657392),
 (8376, 0.96559393),
 (19369, 0.9654488),
 (17071, 0.9654202)]
 </pre>

In [22]:
for i,d in ordered[:10]:
  print(shuffled_gothic.loc[i, 'author'])  #EAP wins with 7 votes - first does not count as it is actual value

EAP
EAP
EAP
EAP
EAP
EAP
EAP
EAP
HPL
HPL


<pre>
EAP
EAP
EAP
EAP
EAP
EAP
EAP
EAP
HPL
HPL
</pre>

##Looking pretty good

Our first attempt looks like a winner. But need to test more.

#Challenge 7

Ok, I am turning you lose on this. We need the equivalent of knn_tester to really check out if our new word-vector version of the crowd is really going to work out. I called my new function vector_knn_tester. Here is signature line:
<pre>
def vector_knn_tester(nlp:spnlp, test_list:list, crowd_matrix:narray, test_labels:list, traing_labels:list, k:int) -> dict:
</pre>
You can see if you match my results.

##But first, let's break out into training and testing

In [0]:
crowd = sentence_vectors[:-1000]  #chop off last 1000 vectors and use remainder as training

In [24]:
training_table = shuffled_gothic[:-1000]
training_labels = training_table['author'].tolist()
print(training_labels[:20])  #['EAP', 'EAP', 'HPL', 'MWS', 'EAP', 'HPL', 'EAP', 'EAP', 'EAP', 'MWS', 'HPL', 'EAP', 'EAP', 'HPL', 'HPL', 'MWS', 'EAP', 'MWS', 'MWS', 'EAP']

['EAP', 'EAP', 'HPL', 'MWS', 'EAP', 'HPL', 'EAP', 'EAP', 'EAP', 'MWS', 'HPL', 'EAP', 'EAP', 'HPL', 'HPL', 'MWS', 'EAP', 'MWS', 'MWS', 'EAP']


In [0]:
gothic_test = gothic_sentences[-1000:]  #use last 1000 sentences for testing

In [26]:
test_table = shuffled_gothic[-1000:]
test_labels = test_table['author'].tolist()
print(test_labels[:20])  #['HPL', 'HPL', 'MWS', 'EAP', 'EAP', 'EAP', 'HPL', 'EAP', 'MWS', 'HPL', 'HPL', 'EAP', 'EAP', 'MWS', 'EAP', 'EAP', 'EAP', 'EAP', 'HPL', 'EAP']

['HPL', 'HPL', 'MWS', 'EAP', 'EAP', 'EAP', 'HPL', 'EAP', 'MWS', 'HPL', 'HPL', 'EAP', 'EAP', 'MWS', 'EAP', 'EAP', 'EAP', 'EAP', 'HPL', 'EAP']


In [0]:
#your code. Should end up defining a function vector_knn_tester with signature:
def vector_knn_tester(nlp:spnlp, test_list:list, crowd_matrix:narray, test_labels:list, training_labels:list, k:int) -> dict:
  cm = {}
  for i in range(len(test_list)):

    top_k = [x for x, y in vector_ordered_distances(nlp, crowd_matrix, test_list[i])[:k]]
    opinions = [training_labels[i] for i in top_k]

    eap_ct = opinions.count('EAP')
    hpl_ct = opinions.count('HPL')
    mws_ct = opinions.count('MWS')

    if (eap_ct >= hpl_ct) & (eap_ct >= mws_ct):
      node = (0, test_labels[i])
    elif (mws_ct > eap_ct) & (mws_ct >= hpl_ct):
      node = (1, test_labels[i])
    elif (hpl_ct > eap_ct) & (hpl_ct > mws_ct):
      node = (2, test_labels[i])
    else:
      print("SHOULDN'T HAPPEN")
      break
    
    if node in cm:
      cm[node] += 1
    else:
      cm[node] = 1
    
  return cm


##Try it on only first 100 sentences

And experiment with different values of k.

In [75]:
%%time

points = vector_knn_tester(nlp, gothic_test[:100], crowd, test_labels, training_labels, 5)  #about a minute at 100

CPU times: user 30.3 s, sys: 39 ms, total: 30.4 s
Wall time: 30.4 s


In [76]:
points

{(0, 'EAP'): 39,
 (0, 'HPL'): 10,
 (0, 'MWS'): 3,
 (1, 'EAP'): 11,
 (1, 'HPL'): 6,
 (1, 'MWS'): 17,
 (2, 'EAP'): 2,
 (2, 'HPL'): 12}

<pre>
{(0, 'EAP'): 39,
 (0, 'HPL'): 10,
 (0, 'MWS'): 3,
 (1, 'EAP'): 11,
 (1, 'HPL'): 6,
 (1, 'MWS'): 17,
 (2, 'EAP'): 2,
 (2, 'HPL'): 12}
 </pre>

In [77]:
(39+17+12)/100  #.68 at k=5

0.68

In [78]:
%%time

points = vector_knn_tester(nlp, gothic_test[:100], crowd, test_labels, training_labels, 11)

CPU times: user 29.3 s, sys: 40 ms, total: 29.3 s
Wall time: 29.3 s


In [79]:
points

{(0, 'EAP'): 37,
 (0, 'HPL'): 8,
 (0, 'MWS'): 5,
 (1, 'EAP'): 12,
 (1, 'HPL'): 6,
 (1, 'MWS'): 14,
 (2, 'EAP'): 3,
 (2, 'HPL'): 14,
 (2, 'MWS'): 1}

<pre>
{(0, 'EAP'): 37,
 (0, 'HPL'): 8,
 (0, 'MWS'): 5,
 (1, 'EAP'): 12,
 (1, 'HPL'): 6,
 (1, 'MWS'): 14,
 (2, 'EAP'): 3,
 (2, 'HPL'): 14,
 (2, 'MWS'): 1}
</pre>

In [80]:
(37+14+14)/100  #.65  with k=11

0.65

In [81]:
%%time

points = vector_knn_tester(nlp, gothic_test[:100], crowd, test_labels, training_labels, 7)

CPU times: user 30.1 s, sys: 30 ms, total: 30.1 s
Wall time: 30.1 s


In [82]:
points

{(0, 'EAP'): 36,
 (0, 'HPL'): 9,
 (0, 'MWS'): 4,
 (1, 'EAP'): 13,
 (1, 'HPL'): 5,
 (1, 'MWS'): 16,
 (2, 'EAP'): 3,
 (2, 'HPL'): 14}

<pre>
{(0, 'EAP'): 36,
 (0, 'HPL'): 9,
 (0, 'MWS'): 4,
 (1, 'EAP'): 13,
 (1, 'HPL'): 5,
 (1, 'MWS'): 16,
 (2, 'EAP'): 3,
 (2, 'HPL'): 14}
</pre>

In [83]:
(36+16+14)/100  #.66  with k=7

0.66

In [84]:
%%time

points = vector_knn_tester(nlp, gothic_test[:100], crowd, test_labels, training_labels, 15)

CPU times: user 28.9 s, sys: 24 ms, total: 28.9 s
Wall time: 28.9 s


In [85]:
points

{(0, 'EAP'): 37,
 (0, 'HPL'): 10,
 (0, 'MWS'): 4,
 (1, 'EAP'): 12,
 (1, 'HPL'): 5,
 (1, 'MWS'): 15,
 (2, 'EAP'): 3,
 (2, 'HPL'): 13,
 (2, 'MWS'): 1}

<pre>
{(0, 'EAP'): 37,
 (0, 'HPL'): 10,
 (0, 'MWS'): 4,
 (1, 'EAP'): 12,
 (1, 'HPL'): 5,
 (1, 'MWS'): 15,
 (2, 'EAP'): 3,
 (2, 'HPL'): 13,
 (2, 'MWS'): 1}
</pre>

In [86]:
(37+15+13)/100  #.65 with k=15

0.65

Looks like 5 is the best from this limited trial. I'll choose 5 and increase to 1000. Took me about 8 minutes.

In [87]:
%%time

points = vector_knn_tester(nlp, gothic_test, crowd, test_labels, training_labels, 5)

CPU times: user 4min 55s, sys: 67.8 ms, total: 4min 55s
Wall time: 4min 55s


In [88]:
points

{(0, 'EAP'): 319,
 (0, 'HPL'): 100,
 (0, 'MWS'): 84,
 (1, 'EAP'): 85,
 (1, 'HPL'): 62,
 (1, 'MWS'): 188,
 (2, 'EAP'): 31,
 (2, 'HPL'): 117,
 (2, 'MWS'): 14}

<pre>
{(0, 'EAP'): 319,
 (0, 'HPL'): 100,
 (0, 'MWS'): 84,
 (1, 'EAP'): 85,
 (1, 'HPL'): 62,
 (1, 'MWS'): 188,
 (2, 'EAP'): 31,
 (2, 'HPL'): 117,
 (2, 'MWS'): 14}
</pre>

In [89]:
(319+188+117)/1000  #.62

0.624

##Hmmmm

In week 4 using Naive Bayes, we got an accuracy of 0.83 on the gothic authors; 20 points higher. We never tried KNN on the full 24K comat from week 6 so don't have that baseline to compare against. My expectation is that the glove version above will be better and certainly faster for KNN. It has a vector size of 300 versus 24K with full comat.

But word vectors are not generally used with KNN. Instead they are used with Artificial Neural Nets. I'd like to turn to that in the next module.

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors. [This paper](https://www.aclweb.org/anthology/Q15-1016.pdf) compares and contrasts the two approaches. (Spoiler: word2vec is slightly better than glove.)
* If there any linguists out there (budding or established), the whole idea of word-vectors representing meaning is a hot topic. You can find an accessible discussion in this 4-part series: [4-part series](https://towardsdatascience.com/beyond-word-embeddings-part-1-an-overview-of-neural-nlp-milestones-82b97a47977f). This group brings neuroimaging into the mix: [Language, Interaction and Computation Laboratory (CLIC)](https://wiki.cimec.unitn.it/tiki-index.php?page=CLIC). And lots more. As I said, a hot-topic.
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/). In this module we used the pre-trained corpus `en_core_web_md`. You can google to see how it was built.
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I have heard that [Annoy](https://pypi.python.org/pypi/annoy) can make these calculations faster, but I have not tried it myself (yet).

##Sorry, have to try one last thing

This ability to do vector arithmetic was highlighted often in early papers with this example:
<pre>
queen=argmax similarity(w,king−man+woman)
</pre>
In other words, we can subtract man from king and add woman and get queen. The argmax says find the most similar. But I'll find the 10 most similar.


In [90]:
#queen=argmax similarity(w,king−man+woman)

king = vec(nlp, 'king')
man = vec(nlp, 'man')
woman = vec(nlp, 'woman')

mystery = addv(subtractv(king,man), woman)

computed_similarities = [(word.text, fast_cosine(mystery, word.vector)) for word in nlp.vocab if word.has_vector]

sorted_similarities = sorted(computed_similarities, key=lambda item: -item[1])
sorted_similarities[:10]



[('King', 0.802426),
 ('KING', 0.802426),
 ('king', 0.802426),
 ('KIng', 0.802426),
 ('Queen', 0.78808445),
 ('QUEEN', 0.78808445),
 ('queen', 0.78808445),
 ('Prince', 0.6401078),
 ('commoner', 0.6401078),
 ('highness', 0.6401078)]

##A bit of an exaggeration

It is true that queen shows up, but king is first. Subsequent work says you have to remove any words in original analogy and only then take the highest.

In [91]:
filtered_similarities = [item for item in sorted_similarities if item[0].lower() not in ['king', 'man', 'woman']]
filtered_similarities[:10]

[('Queen', 0.78808445),
 ('QUEEN', 0.78808445),
 ('queen', 0.78808445),
 ('Prince', 0.6401078),
 ('commoner', 0.6401078),
 ('highness', 0.6401078),
 ('HIGHNESS', 0.6401078),
 ('PRINCE', 0.6401078),
 ('prince', 0.6401078),
 ('Sultan', 0.6401078)]

##Now it works :)

Cute.